# Validate network with only the models

See if the optimization model with only the ml models can generate a feasible solution

This is a relaxing problem

## Root folder and read env variables

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## Load licence gurobi

In [ ]:
##########  LOAD LICENCE GUROBI ##########
import gurobipy as gp

# set env variable with the path of the licence
name_file_licence_gurobi = "gurobi.lic"
path_licence_gurobi = root_path + '\\' + name_file_licence_gurobi
os.environ ["GRB_LICENSE_FILE"] = path_licence_gurobi
print(os.environ["GRB_LICENSE_FILE"])

In [ ]:
######### LAOD CONTENT LICENCE GUROBI #########
with open(path_licence_gurobi, 'r') as f:
    content_licence = f.read()
WLSACCESSID = content_licence.split('\n')[3].split('=')[1] # load WLSACCESSID (string)
WLSSECRET = content_licence.split('\n')[4].split('=')[1] # load WLSSECRET (string)
LICENSEID = int(content_licence.split('\n')[5].split("=")[1]) # load LICENSEID (integer)

params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}

## RUN

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### 1. Load configuration file optimizer and configuration instance to solve
The are principally two kind of files to config optimizer
- **configuration file to create optimizer**: there a files used to create the optimization network such as, list of sets, list of variables, upper bound and lower bound, etc. Pricipally in this files should be parameters that doesn't change too much across the time. For example, the list of variable, is a parameter that if change, the structure of the network change and the machine learning models needs to change too

- **configuration file with instance to solve**: there files that change its values every time that the optimizer solve the problem. It represents the files with the actual values of the features, and so, this values change every time that the optimizer is executed

#### 1.1. IndexTime file

In [ ]:
#################### define set ####################

## paths and files names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_indextime = 'IndexTime.xlsx'
path_indextime = path_folder_config_optimization + file_indextime

# read file
indextime = pd.read_excel(path_indextime)

# set index
index_set_time = pd.Index(indextime['IndexTime'].values)
index_set_time

#### 1.2 Decision variables

In [ ]:
#################### define decision variables ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_allvariables = 'AllVariables.xlsx'
path_allvariables = path_folder_config_optimization + file_allvariables

# read file
config_allvariables = pd.read_excel(path_allvariables)

# table
config_allvariables

#### 1.3 Initial Values
-  This configuration files corresponde to the description **"configuration file with instance to solve"**. This file has the initial values to start the optimizer
-  **OBS: the decision variable that are targets of machine learning models its initial value it nos defined. For all the optimization process since t=0 to t=N all the values are predicted by ml model**

In [ ]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_initvalues = 'InitialValues.xlsx'
path_initvalues = path_folder_config_optimization + file_initvalues

# read file
config_initvalues = pd.read_excel(path_initvalues)

# table
config_initvalues

#### OBS: at this part all the parameters of decision variables and observed variables were defined. Now, it is necesary define a structure of the optimization network to have the capacity to generate whatever network of this kind of problem with the posibilty to change the number of variables, its limits, capacity of tanks and also the NUMBER of process and tanks

#### 1.4 Define models to load
In this file are defined the path to ml models used in each process. Then reading the table the optimizer can go to load the ml model

**As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [ ]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_modelsml = 'ModelsML.xlsx'
path_modelsml = path_folder_config_optimization + file_modelsml

# read file
config_modelsml = pd.read_excel(path_modelsml)

# table
config_modelsml

#### 1.5 Map tanks
Read a configuration table that map each tank in the process. For each tank is mapped the input flows and the output flows

Important, obviosly, the input and outputs of the tank needs to be defined in the table that maps all the features in the network

In [ ]:
#################### define map input outputs each tank ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_maptanks = 'MapTanks.xlsx'
path_maptanks = path_folder_config_optimization + file_maptanks

# read file
config_maptanks = pd.read_excel(path_maptanks)

# table
config_maptanks

#### 1.6 Map process Machine learning models features and target
When each machine learning model is trained, one of the outputs of the tranining process is the artifact with the model and also a master table with the features and target of the model.

- **In this example the individual master table for each model was generated automatically and the sintetized table is generated manually by the author**

- **As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [ ]:
#################### define map features and targer of each model for each process ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_mapprocess_mlmodels = 'MapProcessMLmodels.xlsx'
path_mapprocess_mlmodels = path_folder_config_optimization + file_mapprocess_mlmodels

# read filemapprocess_mlmodels
config_mapprocess_mlmodels = pd.read_excel(path_mapprocess_mlmodels)

# table
config_mapprocess_mlmodels

### 2. Load ML models
Automatically load all the models and save its into a dictionary to call when it is neccesary (as a process can have multiple models, id of the dictionary is the **name_process_model** that indicates the name for a unique model in the optimization network)
- Process A - y1
- Process B - y2
- Process C - y2 (custom model)
- Process B - y3

In [ ]:
config_modelsml

In [ ]:
# process_a_y1
path_model_process_a_y1 = 'artifacts/models/process_a/'
artifact_model_process_a_y1 = 'model.pkl'
path_model_process_a_y1 = path_model_process_a_y1 + artifact_model_process_a_y1
model_process_a_y1 = pd.read_pickle(path_model_process_a_y1)
model_process_a_y1

In [ ]:
# process_b_y2
path_model_process_b_y2 = 'artifacts/models/process_b_y2/'
artifact_model_process_b_y2 = 'model.pkl'
path_model_process_b_y2 = path_model_process_b_y2 + artifact_model_process_b_y2
model_process_b_y2 = pd.read_pickle(path_model_process_b_y2)
model_process_b_y2

In [ ]:
# process_b_y3
path_model_process_b_y3 = 'artifacts/models/process_b_y3/'
artifact_model_process_b_y3 = 'model.pkl'
path_model_process_b_y3 = path_model_process_b_y3 + artifact_model_process_b_y3
model_process_b_y3 = pd.read_pickle(path_model_process_b_y3)
model_process_b_y3

In [ ]:
# process_c_y2
# CUSTOM - read excel table with the parameters

path_model_process_c = 'artifacts/models/process_c/'
artifact_model_process_c = 'model.xlsx'
path_model_process_c = path_model_process_c + artifact_model_process_c
model_process_c = pd.read_excel(path_model_process_c)
model_process_c

### 2. Create gurobi model

In [ ]:
# # create model with licence
env = gp.Env(params=params)
model_opt = gp.Model('Example Optimization Model', env = env)

# create model without licence
# model_opt = gp.Model('Example Optimization Model')

### 3. Create decision variables
It is necesary to have:
- list of elements of the sets of decision var
- table with the list of decision variables to create with important considerations when the are created (for example, upper bound and lower bound)
- **create the decision var and save it into a dictionary of decision var. Use the dictionary to call the decision var when they are used to create a constraints**

#### Aditional, when the decision var is created, set the initial value

- Fix the values of period t=0 for each decision var.

- t=0 represent the actual period or initial period and it in some problems and modelations is kwown

- In addition in this notebook, the values in time t=0 are fixed for all decision variables, inclusive if the decision var have a constraint that define its values in time t = 0 (so, this kind of constraints needs to be defined since t = 1)

In [ ]:
### MANUALLY - create a decision variable with the prefix var_xxx
var_X1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Y1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O4 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Z1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_X2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O5 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O6 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Y2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Y3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_X3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O7 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_TL1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_TL2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_TL3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

### 4. Set initial values decision variables
Set the initial values t=0 for the decision variables that needs this initial values and save in t=0

In [ ]:
# MANUALLY

In [ ]:
# X1
init_value_X1 = 50.0
model_opt.addConstr(var_X1['t0'] == init_value_X1, name = f'Initial Value X0')

# O1
init_value_O1 = 50.0
model_opt.addConstr(var_O1['t0'] == init_value_O1, name = f'Initial Value X0')

# O2	
init_value_O2 = 50.0
model_opt.addConstr(var_O2['t0'] == init_value_O2, name = f'Initial Value X0')

# O3
init_value_O3 = 50.0
model_opt.addConstr(var_O3['t0'] == init_value_O3, name = f'Initial Value X0')

# O4
init_value_O4 = 200.0
model_opt.addConstr(var_O4['t0'] == init_value_O4, name = f'Initial Value X0')

# Z1
#init_value_Z1 = 120 # es una variable que depende de una variable de decision que de hecho deberia ser el mismo valor a menos que baje de nivel el tk
#model_opt.addConstr(var_Z1['t0'] == init_value_Z1, name = f'Initial Value X0')

# X2
init_value_X2 = 5
model_opt.addConstr(var_X2['t0'] == init_value_X2, name = f'Initial Value X0')

# O5
init_value_O5 = 5
model_opt.addConstr(var_O5['t0'] == init_value_O5, name = f'Initial Value X0')

# O6
init_value_O6 = 5
model_opt.addConstr(var_O6['t0'] == init_value_O6, name = f'Initial Value X0')

# X3
init_value_X3 = 5
model_opt.addConstr(var_X3['t0'] == init_value_X3, name = f'Initial Value X0')

# O7
init_value_O7 = 4
model_opt.addConstr(var_O7['t0'] == init_value_O7, name = f'Initial Value X0')

# TL1
init_value_TL1 = 500
model_opt.addConstr(var_TL1['t0'] == init_value_TL1, name = f'Initial Value X0')

# TL2
init_value_TL2 = 500
model_opt.addConstr(var_TL2['t0'] == init_value_TL2, name = f'Initial Value X0')

# TL3
init_value_TL3 = 500
model_opt.addConstr(var_TL3['t0'] == init_value_TL3, name = f'Initial Value X0')

### 5. Lower bound and Upper bound decision variables
For example, lower bound and upper bound of tank level

---
If one decision variables doesn't need a lower bound and upper bound parameter, its value in configuration file is saved as np.NaN

Then filter the configuration table to have only the decision variables that have defined a rate_change

define in differents files one for lower bounds and another for upper bounds

#### 5.1 Lower bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [ ]:
# MANUALLY

In [ ]:
# var_X1
var_X1_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_X1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_X1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Y1
var_Y1_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Y1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Y1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Z1
var_Z1_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Z1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Z1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_X2
var_X2_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_X2,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_X2_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Y2
var_Y2_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Y2,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Y2_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Y3
var_Y3_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Y3,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Y3_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_X3
var_X3_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_X3,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_X3_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_TL1
var_TL1_lower_bound = 100
gppd.add_constrs(model_opt,
                 var_TL1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_TL1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_TL2
var_TL2_lower_bound = 100
gppd.add_constrs(model_opt,
                 var_TL2,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_TL2_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_TL3
var_TL3_lower_bound = 100
gppd.add_constrs(model_opt,
                 var_TL3,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_TL3_lower_bound,  # lower bound
                 name = f'Lower bound X0')

#### 5.2 upper bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [ ]:
##### MANUALLY

In [ ]:
# var_X1
var_X1_upper_bound = 1000
gppd.add_constrs(model_opt,
                 var_X1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_X1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Y1
var_Y1_upper_bound = 400
gppd.add_constrs(model_opt,
                 var_Y1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Y1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Z1
var_Z1_upper_bound = 1000
gppd.add_constrs(model_opt,
                 var_Z1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Z1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_X2
var_X2_upper_bound = 1000
gppd.add_constrs(model_opt,
                 var_X2,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_X2_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Y2
var_Y2_upper_bound = 500
gppd.add_constrs(model_opt,
                 var_Y2,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Y2_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Y3
var_Y3_upper_bound = 450
gppd.add_constrs(model_opt,
                 var_Y3,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Y3_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_X3
var_X3_upper_bound = 1500
gppd.add_constrs(model_opt,
                 var_X3,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_X3_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_TL1
var_TL1_upper_bound = 20000
gppd.add_constrs(model_opt,
                 var_TL1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_TL1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_TL2
var_TL2_upper_bound = 20000
gppd.add_constrs(model_opt,
                 var_TL2,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_TL2_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_TL3
var_TL3_upper_bound = 20000
gppd.add_constrs(model_opt,
                 var_TL3,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_TL3_upper_bound,  # upper bound
                 name = f'Upper bound X0')

### 8. Set values of observed variables
The observed variables are not decision variables, but with the idea to automatize this codes they are defined as decision variables and then a constraint is added to set its values to a unique values

Doing this is a little automatice the creation of the constraints and open the posibility to transform the observed variables into a decision variables and connect other models/tanks/process to this observed variables with a less modification in the codes

In [ ]:
##### MANUALLY

In [ ]:
# var_O1

observed_value_O1 = 50.0
gppd.add_constrs(model_opt,
                 var_O1,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O1,  # value observed variable
                 name = f'set value to observed variable')

In [ ]:
# var_O2

observed_value_O2 = 50.0
gppd.add_constrs(model_opt,
                 var_O2,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O2,  # value observed variable
                 name = f'set value to observed variable')

In [ ]:
# var_O3

observed_value_O3 = 50.0
gppd.add_constrs(model_opt,
                 var_O3,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O3,  # value observed variable
                 name = f'set value to observed variable')

In [ ]:
# var_O4

observed_value_O4 = 200.0
gppd.add_constrs(model_opt,
                 var_O4,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O4,  # value observed variable
                 name = f'set value to observed variable')

In [ ]:
# var_O5

observed_value_O5 = 5.0
gppd.add_constrs(model_opt,
                 var_O5,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O5,  # value observed variable
                 name = f'set value to observed variable')

In [ ]:
# var_O6

observed_value_O6 = 5.0
gppd.add_constrs(model_opt,
                 var_O6,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O6,  # value observed variable
                 name = f'set value to observed variable')

In [ ]:
# var_O7

observed_value_O7 = 4.0
gppd.add_constrs(model_opt,
                 var_O7,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O7,  # value observed variable
                 name = f'set value to observed variable')

### 8. Load a Machine Learning Model as constraints that represent the relations in the process
Add a constraint defined as a machine learning model that represent the relation between X1, X2 with the output of the process (Y1)

In [ ]:
############# Connect model process_a_y1

# Generate instance of machine learning model
instance = pd.DataFrame([var_X1, var_O1, var_O2, var_O3]).T
instance.columns = ["X1", "O1", "O2", "O3"]

# Add machine learning model as constraint

pred_constr_ml_process_a_y1 = add_predictor_constr(gp_model = model_opt, 
                                                   predictor = model_process_a_y1, 
                                                   input_vars = instance, # instance pandas gurobi
                                                   output_vars = var_Y1, # target
                                                   name = f'model_predict Y1'
                                                  )

pred_constr_ml_process_a_y1.print_stats()

In [ ]:
############# Connect model model_process_b_y2


# Generate instance of machine learning model
instance = pd.DataFrame([var_Z1, var_X2, var_O5, var_O6]).T
instance.columns = ["Z1", "X2", "O5", "O6"]

# Add machine learning model as constraint

pred_constr_ml_model_process_b_y2 = add_predictor_constr(gp_model = model_opt, 
                                                   predictor = model_process_b_y2, 
                                                   input_vars = instance, # instance pandas gurobi
                                                   output_vars = var_Y2, # target
                                                   name = f'model_predict Y2'
                                                  )

pred_constr_ml_model_process_b_y2.print_stats()

In [ ]:
############# Connect model model_process_b_y3


# Generate instance of machine learning model
instance = pd.DataFrame([var_Z1, var_X2]).T
instance.columns = ["Z1", "X2"]

# Add machine learning model as constraint

pred_constr_ml_model_process_b_y3 = add_predictor_constr(gp_model = model_opt, 
                                                   predictor = model_process_b_y3, 
                                                   input_vars = instance, # instance pandas gurobi
                                                   output_vars = var_Y3, # target
                                                   name = f'model_predict Y3'
                                                  )

pred_constr_ml_model_process_b_y3.print_stats()

In [ ]:
var_X2

### 8. Define a custom function as constraints that represent the relations in the process C

**Important: this constraint and the objective functions are more hardoded that the rest of constraints**

In [ ]:
# define parameters of the constraint
alpha_feature_1 = 1/5
alpha_feature_2 = 15

In [ ]:
# define function as constraint
gppd.add_constrs(model_opt, 
                 (alpha_feature_1 * var_X3 + alpha_feature_2 * var_O7), 
                 gp.GRB.EQUAL, 
                 var_Y2, 
                 name = 'function as constraint output process C'
                )

In [ ]:
model_opt.update()

In [ ]:
model_opt

### 9. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
var_Y3.sum()

In [ ]:
var_Y2.sum()

In [ ]:
# optization
model_opt.setObjective(var_Y1.sum() + var_Y2.sum() + var_Y3.sum(),
                       gp.GRB.MAXIMIZE)

## ----> return the mdoel_opt that has defined for this instance can get the solution <----

In [ ]:
model_opt

### 10. Optimize and get optimal values

In [ ]:
# solve
model_opt.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
model_opt.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################
solution["var_X1"] = var_X1.gppd.X
solution["var_O1"] = var_O1.gppd.X
solution["var_O2"] = var_O2.gppd.X
solution["var_O3"] = var_O3.gppd.X
solution["var_Y1"] = var_Y1.gppd.X
solution["var_O4"] = var_O4.gppd.X
solution["var_Z1"] = var_Z1.gppd.X
solution["var_X2"] = var_X2.gppd.X
solution["var_O5"] = var_O5.gppd.X
solution["var_O6"] = var_O6.gppd.X
solution["var_Y2"] = var_Y2.gppd.X
solution["var_Y3"] = var_Y3.gppd.X
solution["var_X3"] = var_X3.gppd.X
solution["var_O7"] = var_O7.gppd.X
solution["var_TL1"] = var_TL1.gppd.X
solution["var_TL2"] = var_TL2.gppd.X
solution["var_TL3"] = var_TL3.gppd.X

######################## # get value objetive function ########################
opt_objetive_function = model_opt.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
solution

In [ ]:
stop

In [ ]:
# var_y2 = 100
# var_x3 = 200
# var_o7 = 4
200 * 1/5 + 4*15 # output tag y2

In [ ]:
# probar que el modelo y3 funciona bien
instance_debugging_b_y3 = [[2250, 0.0]]  # z1, x2
model_process_b_y3.predict(instance_debugging_b_y3)

In [ ]:
var_Z1

In [ ]:
var_Z1, var_X2

In [ ]:
# tabla de limites operacionales
config_allvariables.dropna()